<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/supertrend_tarama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import yfinance as yf
import math
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


def calculate_supertrend(df, atr_period, multiplier):


    high = df['High']
    low = df['Low']
    close = df['Close']


    price_diffs = [high - low,
                   high - close.shift(),
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    atr = true_range.ewm(alpha=1/atr_period, min_periods=atr_period).mean()


    hl2 = (high + low) / 2

    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)


    supertrend = [True] * len(df)

    for i in range(1, len(df.index)):
        curr, prev = i, i-1


        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True

        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False

        else:
            supertrend[curr] = supertrend[prev]


            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]


        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan

    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)


atr_period = 10
atr_multiplier = 3.0



turkish_stock_symbols = [
    "ACSEL.IS", "ADESE.IS", "AFYON.IS", "AGHOL.IS", "AKBNK.IS",
    "AKCNS.IS", "AKENR.IS", "AKFGY.IS", "AKGRT.IS", "AKSA.IS",
    "AKSEN.IS", "AKSGY.IS", "AKSUE.IS", "ALARK.IS", "ALBRK.IS",
    "ALGYO.IS", "ALKA.IS", "ALKIM.IS",
    "ANELE.IS", "ANHYT.IS", "ARCLK.IS", "ARENA.IS",
    "ARSAN.IS", "ASELS.IS",  "ATAGY.IS", "ATEKS.IS",
    "ATSYH.IS", "AVGYO.IS", "AVHOL.IS", "AVOD.IS", "AVTUR.IS",
    "AYCES.IS", "AYEN.IS", "AYGAZ.IS", "BAGFS.IS", "BAKAB.IS",
     "BANVT.IS", "BASCM.IS", "BAYRK.IS",
    "BERA.IS", "BEYAZ.IS", "BFREN.IS", "BIMAS.IS", "BJKAS.IS",
    "BLCYT.IS", "BNTAS.IS",  "BOSSA.IS",
    "BRISA.IS", "BRKO.IS", "CASA.IS", "CCOLA.IS", "CELHA.IS",
    "CEMAS.IS", "CEMTS.IS", "CIMSA.IS", "CLEBI.IS", "CRDFA.IS",
    "CRDFA.IS", "DESA.IS",
    "DMSAS.IS", "DOAS.IS", "DOHOL.IS",  "DURDO.IS",
    "ECILC.IS", "ECZYT.IS", "EDIP.IS",  "EGEEN.IS",
    "EGPRO.IS", "EGSER.IS",  "EKGYO.IS", "EMKEL.IS",
    "ENJSA.IS", "ENKAI.IS",   "EREGL.IS",
    "ERSU.IS", "ESCOM.IS", "ESEN.IS", "ETILR.IS",
    "FENER.IS", "FROTO.IS", "GARAN.IS", "GEDZA.IS", "GENTS.IS",
    "GEREL.IS", "GLYHO.IS", "GLRYH.IS", "GOLTS.IS", "GOODY.IS",
    "GOZDE.IS", "GRNYO.IS", "GSDDE.IS", "GSRAY.IS", "GUBRF.IS",
    "HALKB.IS", "HATEK.IS", "HEKTS.IS",
    "HLGYO.IS", "HURGZ.IS", "IHLGM.IS", "IHLGM.IS", "IPEKE.IS",
    "ISBIR.IS", "ISCTR.IS", "ISFIN.IS", "ISGSY.IS", "ISGYO.IS",
    "ISYAT.IS",  "IZMDC.IS",  "JANTS.IS",
    "KAPLM.IS", "KAREL.IS", "KARSN.IS", "KARTN.IS", "KATMR.IS",
    "KCHOL.IS", "KERVT.IS", "KLGYO.IS", "KLNMA.IS", "KLMSN.IS",
    "KONYA.IS", "KORDS.IS", "KRDMD.IS", "KSTUR.IS",
    "KUTPO.IS", "LKMNH.IS", "LOGO.IS", "MAALT.IS", "MAKTK.IS",
    "MARKA.IS", "MARTI.IS", "MERKO.IS", "MGROS.IS", "NATEN.IS",
    "NETAS.IS", "NIBAS.IS", "NTHOL.IS", "NUGYO.IS", "ODAS.IS",
     "ORGE.IS", "OSMEN.IS",
    "OTKAR.IS", "OYAKC.IS", "OYLUM.IS",
     "PAGYO.IS", "PARSN.IS", "PETKM.IS", "PETUN.IS",
    "PGSUS.IS", "PINSU.IS", "PKART.IS",  "PNSUT.IS",
    "PRKME.IS", "PRZMA.IS", "PSDTC.IS", "RAYSG.IS",
    "RODRG.IS",  "RTALB.IS", "RYGYO.IS", "RYSAS.IS",
     "SANEL.IS", "SARKY.IS", "SASA.IS",
    "SEKFK.IS", "SELEC.IS", "SEYKM.IS", "SISE.IS", "SKBNK.IS",
    "SKTAS.IS", "SMART.IS", "SNGYO.IS", "SNPAM.IS",
    "SOKM.IS", "SOKM.IS", "SOKM.IS", "SOKM.IS", "SOKM.IS",
    "SOKM.IS", "SONME.IS", "SRVGY.IS",
     "SAHOL.IS", "TATGD.IS", "TAVHL.IS", "TAVHL.IS",
    "TBORG.IS", "TCELL.IS", "TDGYO.IS", "TGSAS.IS", "THYAO.IS",
    "TKFEN.IS", "TMSN.IS", "TOASO.IS",
     "TRCAS.IS", "TRGYO.IS",
    "TSKB.IS", "TSPOR.IS", "TTRAK.IS", "TUKAS.IS", "TUPRS.IS",
    "TUREX.IS"]

supertrend_stocks = []

# Her bir Türk hissesi için döngü
for symbol in turkish_stock_symbols:
    try:

        df = yf.download(symbol, period="1mo", interval="5m", threads=False) # PERİYOT VERİLERİ = 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo


        if len(df) == 0:
            continue

        # Atr_period ve atr_multiplier'ı istediğiniz değerlerle değiştirin
        atr_period = 10
        atr_multiplier = 3


        supertrend = calculate_supertrend(df, atr_period, atr_multiplier)


        if not supertrend['Supertrend'][-2] and supertrend['Supertrend'][-1]:
            supertrend_stocks.append(symbol)
    except Exception as e:
        print(f"{symbol} işlenirken hata oluştu: {e}")


# Supertrend koşullarını karşılayan hisselerin listesini yazdır
df_supertrend = pd.DataFrame(supertrend_stocks, columns=["Supertrend Koşullarını Karşılayan Hisseler"])
print(df_supertrend)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

   Supertrend Koşullarını Karşılayan Hisseler
0                                     AVOD.IS
1                                    BANVT.IS
2                                    BOSSA.IS
3                                    FROTO.IS
4                                    KARTN.IS
5                                    KLNMA.IS
6                                    KONYA.IS
7                                    KSTUR.IS
8                                    PRKME.IS
9                                    TBORG.IS
10                                    TMSN.IS
